In [1]:
import sys
import os
src_dir = os.path.join('/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/', 'src')
sys.path.append(src_dir)

import pandas as pd
from preprocessing import collection_preprocessing
from retrieval.sparse_retrieval import bm25

In [2]:
collected = pd.read_csv('reference_urls_collected.csv')
questions = pd.read_csv('questions_updated_urls.csv')

Make collection into list of (url, content)

In [3]:
collected_clean = collected[collected['Exception'].isnull()]

In [4]:
questions_clean = collection_preprocessing.remove_null_questions(questions, collected)

In [5]:
questions_clean.head()

,Unnamed: 0,Year,Month,Question,Answer,Document,URLs,Cleaned_URLs
0,0,2020,9,\n \n10. Welke redenatie wordt gevolgd door he...,\nKleine kinderen hebben inderdaad te maken m...,https://amsterdam.raadsinformatie.nl/document/...,['https://www.rivm.nl/coronavirus-covid-19/ond...,[https://www.rivm.nl/coronavirus-covid-19/onde...
1,1,2020,6,\n7.\nKan het college de reeds bestaande zwemp...,\nVoor het vinden van de officiële zwemplekken...,https://amsterdam.raadsinformatie.nl/document/...,['https://www.amsterdam.nl/veelgevraagd/?casei...,[https://www.amsterdam.nl/veelgevraagd/?caseid...
2,3,2021,8,\n \n3. Erkent het college daarom dat vergunni...,\nBij alle windturbines die in Amsterdam zij...,https://amsterdam.raadsinformatie.nl/document/...,['https://www.rijksoverheid.nl/documenten/kame...,[https://www.rijksoverheid.nl/documenten/kamer...
3,4,2021,8,\n6. De overheid heeft een in de Grondwet vera...,\nDe Raad van State en het Rijk hebben aange...,https://amsterdam.raadsinformatie.nl/document/...,['https://www.rijksoverheid.nl/documenten/kame...,[https://www.rijksoverheid.nl/documenten/kamer...
4,5,2019,10,\n \n \n \n ...,\nHet college heeft contact met woningcorpora...,https://amsterdam.raadsinformatie.nl/document/...,['https://www.ggd.amsterdam.nl/gezond-wonen/lo...,[https://www.ggd.amsterdam.nl/gezond-wonen/lod...


In [6]:
document_list = []

for index, row in collected_clean.iterrows():
    document_id = row['URL']
    document_content = row['Textual_Content']
    document_tuple = (document_id, document_content)
    document_list.append(document_tuple)


TF-IDF Ranking

In [7]:
from retrieval.sparse_retrieval.tfidf import tfidf_search

In [8]:
questions_clean['Question'][80]

'\n \n4. Welke acties gaat het college op korte termijn ondernemen om het per auto halen \nen brengen van treinreizigers per ommegaande te verbeteren, zowel wat betreft \nde bereikbaarheid als het legaal kunnen stoppen om mensen veilig in en uit te \nlaten stappen zonder overlast te veroorzaken? \n \n'

In [66]:
questions_clean['Question'][1]

'\n7.\nKan het college de reeds bestaande zwemplekken in Amsterdam en de directe \nomgeving, die zich op fietsafstand bevinden, beter communiceren zodat mensen \nweten waar ze allemaal heen kunnen op de fiets om te zwemmen?\n'

In [65]:
sorted1 = tfidf_search(questions_clean['Question'][1], document_list)

In [74]:
questions_clean['URLs'][1]

"['https://www.amsterdam.nl/veelgevraagd/?caseid=%7BD6E280FB-4A76-40A0-9B88-12B87E446FA6%7D', 'https://www.ggd.amsterdam.nl/gezond-wonen/zwemmen-open-water/']"

In [77]:
questions_clean[questions_clean['Question']=="""\n7.\nKan het college de reeds bestaande zwemplekken in Amsterdam en de directe \nomgeving, die zich op fietsafstand bevinden, beter communiceren zodat mensen \nweten waar ze allemaal heen kunnen op de fiets om te zwemmen?\n"""]['Cleaned_URLs']

1    [https://www.amsterdam.nl/veelgevraagd/?caseid...
Name: Cleaned_URLs, dtype: object

In [69]:
all_results['\n7.\nKan het college de reeds bestaande zwemplekken in Amsterdam en de directe \nomgeving, die zich op fietsafstand bevinden, beter communiceren zodat mensen \nweten waar ze allemaal heen kunnen op de fiets om te zwemmen?\n']

[('https://www.ggd.amsterdam.nl/gezond-wonen/zwemmen-open-water/',
  0.29021599448385266),
 ('https://www.amsterdam.nl/verkeer-vervoer/fiets/fietsdepot/fiets-graveren/',
  0.2485345844654728),
 ('https://www.amsterdam.nl/veelgevraagd/?caseid=%7BD6E280FB-4A76-40A0-9B88-12B87E446FA6%7D',
  0.24423157836116355),
 ('https://www.amsterdam.nl/publish/pages/931968/raadsbrief_positief_perspectief_-_weerbaar_opgroeien_in_amsterdam.pdf',
  0.20228964476543565),
 ('https://www.amsterdam.nl/zorg-ondersteuning/ondersteuning/vluchtelingen/24-uursopvang-ongedocumenteerden/',
  0.199519651860172),
 ('https://www.amsterdam.nl/wonen-leefomgeving/duurzaam-amsterdam/windmolens-amsterdam/reflectiefase/',
  0.17672586923573547),
 ('https://www.amsterdam.nl/wonen-leefomgeving/duurzaam-amsterdam/windmolens-amsterdam/klankbordgroepen-windmolens-amsterdam/',
  0.17286856465672704),
 ('https://www.rivm.nl/windenergie/windturbines-gezondheid/vragen-en-antwoorden-gezondheidseffecten-windturbines',
  0.168051995306

Perform tf_idf to all

In [51]:
def perform_tfidf_search(queries, collection):
    """
    Perform TF-IDF search for each query in a list of queries.
    Input:
        queries - a list of queries
        collection: a list of tuples (document_id, document_content)
    Output: a dictionary where the key is the query and the value is a list of (document_id, score) tuples
    """
    search_results = {}

    for query in queries:
        results = tfidf_search(query, collection)
        search_results[query] = results

    return search_results

In [53]:
all_results = perform_tfidf_search(list(questions_clean['Question']), document_list)

In [132]:
all_results

{'\n \n10. Welke redenatie wordt gevolgd door het RIVM c.q. de GGD dat de besmetting \ndoor kinderen bij een SARS-virus anders is dan andere griepvirussen? \n \n': [('https://www.rivm.nl/windenergie/reactie-artikel-BB',
   0.22174630665759476),
  ('https://www.rivm.nl/bibliotheek/rapporten/680300007.pdf',
   0.172320244970176),
  ('https://www.rivm.nl/windenergie/windturbines-gezondheid/vragen-en-antwoorden-gezondheidseffecten-windturbines',
   0.1720493239668803),
  ('https://www.rivm.nl/mpox-apenpokken', 0.17053423061516645),
  ('https://www.ggd.amsterdam.nl/gezond-wonen/rook-roet-gezondheid/houtstook/toename-gemelde-overlast-houtstook-2021/',
   0.1493548884044095),
  ('https://www.rivm.nl/coronavirus-covid-19/onderzoek', 0.14676846016099315),
  ('https://www.rivm.nl/nieuws/brede-beschikbaarheid-van-zelftesten-verhoogt-testbereidheid',
   0.1459871365800816),
  ('https://www.ggd.amsterdam.nl/coronavirus/testen-op-corona/',
   0.1455426028395028),
  ('https://www.rivm.nl/sites/defaul

In [128]:
import pickle

In [129]:
with open ('tfidf_results.pickle', 'wb') as f:
    pickle.dump(all_results, f)

Evaluate

In [22]:
from retrieval.evaluation import evaluate

In [58]:
questions_clean.loc[questions_clean['Question'] == """\n \n10. Welke redenatie wordt gevolgd door het RIVM c.q. de GGD dat de besmetting \ndoor kinderen bij een SARS-virus anders is dan andere griepvirussen? \n \n"""]['Cleaned_URLs']

0    [https://www.rivm.nl/coronavirus-covid-19/onde...
Name: Cleaned_URLs, dtype: object

# Calculate Recall aggain 

In [91]:
def dataframe_to_dict(df):
    """
    Convert a DataFrame with questions and URLs into a dictionary.
    Input:
        df: a pandas DataFrame with columns 'question' and 'urls'
    Output: a dictionary where the key is the question and the value is a list of URLs
    """
    question_urls_dict = {}

    for _, row in df.iterrows():
        question = row['Question']
        urls = row['Cleaned_URLs']

        question_urls_dict[question] = urls

    return question_urls_dict


In [97]:
relevant_docs = dataframe_to_dict(questions_clean)

In [122]:
def calculate_recall_at_k(queries, collection, relevant_docs, k_values):
    """
    Calculate Recall@K for a list of queries and specified values of k.
    Input:
        queries - a list of queries
        collection: a list of tuples (document_id, document_content)
        relevant_docs: a dictionary where the key is the query and the value is a set of relevant document IDs
        k_values: a list of k values for recall calculation
    Output: a dictionary where the key is the query and the value is a dictionary of recall values at each k
    """
    search_results = perform_tfidf_search(queries, collection)
    recall_results = {}

    for query in queries:
        results = search_results[query]
        relevant = relevant_docs[query]

        recall_values = {}
        for k in k_values:
            recall = evaluate.recall_k(results, relevant, k)
            recall_values[k] = recall

        recall_results[query] = recall_values

    return recall_results


In [101]:
recalls = calculate_recall_at_k(list(questions_clean['Question']), document_list, relevant_docs, [1,5,10])

In [105]:
def calculate_average_recall(recall_results):
    """
    Calculate the average recall values per recall type from a dictionary of recall values.
    Input:
        recall_results: a dictionary where the key is the query and the value is a dictionary of recall values at each k
    Output: a dictionary where the key is the recall type and the value is the average recall value
    """
    average_recall = {}

    for query_recall in recall_results.values():
        for k, recall in query_recall.items():
            if k not in average_recall:
                average_recall[k] = 0.0
            average_recall[k] += recall

    query_count = len(recall_results)
    for k in average_recall:
        average_recall[k] /= query_count

    return average_recall


In [106]:
calculate_average_recall(recalls)

{1: 0.16544117647058823, 5: 0.2610294117647059, 10: 0.37009803921568624}

# Random Retrieval

In [107]:
from retrieval.random_retrieval import random_retrieve

In [108]:
random_result = random_retrieve.retrieve_document(document_list)

In [111]:
def perform_random_search(queries, collection):
    """
    Perform TF-IDF search for each query in a list of queries.
    Input:
        queries - a list of queries
        collection: a list of tuples (document_id, document_content)
    Output: a dictionary where the key is the query and the value is a list of (document_id, score) tuples
    """
    search_results = {}

    for query in queries:
        results = random_retrieve.retrieve_document(collection)
        search_results[query] = results

    return search_results

In [117]:
random_result = perform_random_search(list(questions_clean['Question']), document_list)

In [123]:
import random

def calculate_recall_at_k_random(queries, collection, relevant_docs, k_values):
    """
    Calculate Recall@K for a list of queries and specified values of k.
    Input:
        queries - a list of queries
        collection: a list of tuples (document_id, document_content)
        relevant_docs: a dictionary where the key is the query and the value is a set of relevant document IDs
        k_values: a list of k values for recall calculation
    Output: a dictionary where the key is the query and the value is a dictionary of recall values at each k
    """
    recall_results = {}
    random.seed(42)
    for query in queries:
        random.shuffle(collection)  # Shuffle the collection randomly

        results = collection[:max(k_values)]  # Retrieve documents randomly
        relevant = relevant_docs[query]

        recall_values = {}
        for k in k_values:
            recall = evaluate.recall_k(results, relevant, k)
            recall_values[k] = recall

        recall_results[query] = recall_values

    return recall_results


In [124]:
random_recalls = calculate_recall_at_k_random(list(questions_clean['Question']), document_list, relevant_docs, [1,5,10])

In [125]:
calculate_average_recall(random_recalls)

{1: 0.007352941176470588, 5: 0.029411764705882353, 10: 0.09191176470588236}

Get average

In [79]:
def calculate_average(numbers):
    """
    Calculate the average of a list of numbers.
    Input:
        numbers - a list of numbers
    Output: the average of the numbers
    """
    if not numbers:
        return 0  # Return 0 if the list is empty to avoid division by zero

    total = sum(numbers)
    average = total / len(numbers)
    return average


In [81]:
recall_scores_5

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

# Passage experimentation

In [3]:
questions_clean = collection_preprocessing.remove_null_questions(questions, collected)

In [4]:
collected_with_passages = collection_preprocessing.get_passages(collected)
passages = collection_preprocessing.id_passages(collected_with_passages)

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/src/preprocessing/collection_preprocessing.py:14: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, passage in df['Passages'].iteritems():


In [5]:
queries = list(questions['Question'])

In [6]:
result = bm25.bm25_elasticsearch(passages, queries)

In [7]:
result

[]

In [7]:
def average_token_length(strings):
    total_length = 0
    total_tokens = 0

    for string in strings:
        tokens = string.split()
        total_length += sum(len(token) for token in tokens)
        total_tokens += len(tokens)

    if total_tokens > 0:
        average_length = total_length / total_tokens
        return average_length
    else:
        return 0  # To handle the case when the list is


In [11]:
collected_with_passages['Textual_Content'][0]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCorona-onderzoek | RIVM\n\n\n\n\n\n\n\n\n\n\n\nOverslaan en naar de inhoud gaan\nDirect naar de hoofdnavigatie\n\n\n\n\n\n \n \nRijksinstituut voor Volksgezondheiden Milieu\nMinisterie van Volksgezondheid,Welzijn en Sport\n\n\n\n\n\n\n\n\nNederlands\n\n\nEnglish\n\n\n\n\n\n\n\nRIVM De zorg van morgen begint vandaag \n          \n\n\n\n\n\n\nMenuMenu ingeklapt\n\n\n\n\nHome\n\n\nOnderwerpen\n\n\nOver RIVM\n\n\nPublicaties\n\n\nInternationaal\n\n\nContact\n\n\nAgenda\n\n\n\n\nNederlands\n\n\nEnglish\n\n\n\nZoekformulier ingeklapt\n\n\n\n\n\n\nZoeken\n\n\n\n\n\nZoeken\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\nCOVID-19\n\n\n              Corona-onderzoek\n          \n\n Corona-onderzoek\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n× In dit onderwerp Menu\n\n\n\n\n\n\n\nWeekcijfers\n\n\nActuele informatie\n\n\nResponsteam COVID-19\n\n\n\n\nVragen en achtergronden\n\n\nOutbreak Management Team (OMT)\n\n\nQuarantaine en isolatie\n\n\nRisicogroepen en COVID-19\n\n\nVarianten 

In [13]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

: 

: 